#### Leitura dos arquivos sobre o clima

Arquivos baixados do site [inmet](https://portal.inmet.gov.br/dadoshistoricos) (arquivos de 2022)

In [1]:
import datetime as dt
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import isnan

REMOVE_COLUMNS_REGION = ["CODIGO (WMO):", "DATA DE FUNDACAO:"]
GET_COLUMNS_CLIMATE = [
    "Data",
    "Hora UTC",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    "VENTO, VELOCIDADE HORARIA (m/s)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    "RADIACAO GLOBAL (Kj/m²)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
]

%matplotlib inline


In [2]:
climate_dir = "./data/2022"
list_df_station = []
list_df_climate = []

count = 0

for file_name in os.listdir(climate_dir):
    # Nossa tabela fato, utiliza o nome da UF, latitude, longitude e altitude
    df_station = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        nrows=7,
    ).set_index("REGIAO:").transpose().reset_index(drop=True)

    df_station = df_station.drop(columns=REMOVE_COLUMNS_REGION)

    df_station = df_station.rename(columns={"UF:":"UF",
                                            "LATITUDE:":"latitude",
                                            "LONGITUDE:":"longitude",
                                            "ALTITUDE:":"altitude",
                                            "ESTACAO:": "cidade"})


    df_climate = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        skiprows=8,
    ).dropna(how="all", subset=GET_COLUMNS_CLIMATE[2:]).get(GET_COLUMNS_CLIMATE)

    df_climate = df_climate.rename_axis("id")

    df_climate = df_climate.rename(columns={"CODIGO (WMO)": "id",
                                  "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipTotalHorario",
                                  "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "tempAr",
                                  "VENTO, VELOCIDADE HORARIA (m/s)": "ventoVeloHoraria",
                                  "RADIACAO GLOBAL (Kj/m²)": "radGlobal",
                                  "UMIDADE RELATIVA DO AR, HORARIA (%)": "umidRelAr"})

    # Converter as colunas relevantes para tipo numérico
    numeric_columns = ['precipTotalHorario', 'ventoVeloHoraria', 'tempAr', 'radGlobal', 'umidRelAr']

    for column in numeric_columns:
        df_climate[column] = pd.to_numeric(
            df_climate[column].apply(lambda v: str(v).replace(',', '.') if not type(v) == float else v),
            errors='raise'
        )

        # Calcular média da coluna para substituir os valores nan
        column_mean = df_climate[column].dropna().mean()
        if isnan(column_mean):
            column_mean = 0
        df_climate[column] = df_climate[column].fillna(column_mean)

    #Adicionando id da FK
    df_climate["id_regiao"] = count
    count = count+1



    list_df_station.append(df_station)
    list_df_climate.append(df_climate)

In [3]:
df_all_station = pd.concat(list_df_station, ignore_index=True)
df_all_station = df_all_station.rename_axis("id")
df_all_station.to_csv("./data/dim_regiao.csv", sep=";")

In [4]:
df_all_climate = pd.concat(list_df_climate, ignore_index=True)

In [5]:
# Criar timestamp usando as colunas 'Data' e 'Hora UTC'
def create_timestamp(row):
    return dt.datetime.strptime(f"{row['Data']} {row['Hora UTC']}", "%Y/%m/%d %H%M UTC")

df_all_climate["timestamp"] = df_all_climate.apply(create_timestamp, axis=1)
df_all_climate = df_all_climate.drop(["Data", "Hora UTC"], axis=1)

In [6]:
df_all_climate = df_all_climate.rename_axis("id")
df_all_climate.to_csv("./data/fato_dados_meteorologicos.csv", sep=";")
df_all_climate

,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp
id,,,,,,,
0,0.0,1.9,19.2,1497.154553,89.0,0,2022-01-01 00:00:00
1,0.4,2.1,19.0,1497.154553,92.0,0,2022-01-01 01:00:00
2,0.0,2.6,19.1,1497.154553,91.0,0,2022-01-01 02:00:00
3,0.0,2.8,18.8,1497.154553,91.0,0,2022-01-01 03:00:00
4,0.0,1.3,18.5,1497.154553,92.0,0,2022-01-01 04:00:00
...,...,...,...,...,...,...,...
3437918,0.0,1.9,28.5,2927.400000,40.0,566,2022-12-31 19:00:00
3437919,0.0,2.8,26.9,1222.400000,44.0,566,2022-12-31 20:00:00
3437920,0.0,3.9,26.6,518.700000,50.0,566,2022-12-31 21:00:00


In [7]:
df_grouped_climate = df_all_climate.set_index('timestamp')
df_grouped_climate = df_grouped_climate.groupby("timestamp").mean()
df_grouped_climate

,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao
timestamp,,,,,,
2022-01-01 00:00:00,0.474844,1.607250,22.596342,1192.919960,81.318823,322.746914
2022-01-01 01:00:00,0.306536,1.606084,22.205653,1172.373779,83.259537,318.950156
2022-01-01 02:00:00,0.345876,1.565165,21.821098,1172.787725,84.494296,318.386997
2022-01-01 03:00:00,0.230854,1.532500,21.616310,1189.554431,85.013952,317.879154
2022-01-01 04:00:00,0.169650,1.534942,21.320228,1192.054759,86.043409,318.220183
...,...,...,...,...,...,...
2022-12-31 19:00:00,0.233127,2.292636,28.312385,1699.724795,58.892438,296.524838
2022-12-31 20:00:00,0.270761,2.170969,27.553467,1122.119650,62.001490,298.424508
2022-12-31 21:00:00,0.225741,1.874457,26.447883,568.365166,66.769132,298.385965
